In [1]:
! pip install datasets
! pip install transformers
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires 

In [2]:
# Import Libraries
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer
import torch

## Load dataset

In [3]:
# Load the dataset
dataset = load_dataset("rajpurkar/squad")

# Check the structure
print(dataset)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')
tokenizer.add_special_tokens({'bos_token':'[BOS]'})
tokenizer.add_special_tokens({'eos_token':'[EOS]'})
tokenizer.add_special_tokens({'additional_special_tokens':['[CON]','[QUE]','[ANS]']})

# Check the tokenizer
print(tokenizer.special_tokens_map)
print(tokenizer.all_special_ids)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'bos_token': '[BOS]', 'eos_token': '[EOS]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['[CON]', '[QUE]', '[ANS]']}
[30522, 30523, 100, 102, 0, 101, 103, 30524, 30525, 30526]


In [ ]:
def map_fn(example):
  # Tokenize the context, question, and answer
  tokenized_context = tokenizer.tokenize(example['context'])
  tokenized_question = tokenizer.tokenize(example['question'])
  tokenized_answer = tokenizer.tokenize(example['answers']['text'][0])

  # Create the input and output token in terms of the following structure
  input = ['[CON]'] + tokenized_context + ['[QUE]'] + tokenized_question + ['[ANS]'] + tokenized_answer
  output = tokenized_context + ['[QUE]'] + tokenized_question + ['[ANS]'] + tokenized_answer + ['[EOS]']
  ans_index = input.index('[ANS]')

  # Convert the tokens to ids
  input = tokenizer.convert_tokens_to_ids(input)
  output = tokenizer.convert_tokens_to_ids(output)

  return {'input':input, 'output':output, 'ans_index': ans_index}

def filter_fn(example, max_length=512):
  # Filter out the examples that are too long
  return len(example['input']) <= max_length

# Adopt the previous function
dataset['train'] = dataset['train'].map(map_fn)
dataset['train'] = dataset['train'].remove_columns(['id','title','context','question','answers'])
dataset['train'] = dataset['train'].filter(filter_fn)
dataset['validation'] = dataset['validation'].map(map_fn)
dataset['validation'] = dataset['validation'].remove_columns(['id','title','context','question','answers'])
dataset['validation'] = dataset['validation'].filter(filter_fn)
dataset['train'],dataset['validation']

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (718 > 512). Running this sequence through the model will result in indexing errors


Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10570 [00:00<?, ? examples/s]

(Dataset({
     features: ['input', 'output', 'ans_index'],
     num_rows: 87479
 }),
 Dataset({
     features: ['input', 'output', 'ans_index'],
     num_rows: 10512
 }))

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Merge the dataset
merged_dataset = concatenate_datasets([dataset['train'], dataset['validation']])

# Push to Hugging Face Hub
merged_dataset.push_to_hub('Wodeyuanbukongda/SQuard_Chatbot_Llama')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/98 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Wodeyuanbukongda/SQuard_Chatbot_Llama/commit/a22e2c558a83ff2ebb3c8083801d6f020c8d0c03', commit_message='Upload dataset', commit_description='', oid='a22e2c558a83ff2ebb3c8083801d6f020c8d0c03', pr_url=None, pr_revision=None, pr_num=None)